In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class TraficLights(tf.keras.layers.Layer):
    def __init__(self, opts, t0=5, tf=15):
        super().__init__()
        self.opts = opts # numero de casos de semaforos considerados
        self.t0 = t0 # rango inferior de la ventana de tiempo
        self.tf = tf # rango superior de la ventana de tiempo
    
    def get_config(self):
        return {'opts': self.opts}

    def build(self, input_shape): # (batch, units)
        self.Wopt = self.add_weight(shape=(input_shape[-1], self.opts)) # matriz de pesos para opciones de control
        self.bopt = self.add_weight(shape=(self.opts,)) # bias para opciones de control

        self.Wt = self.add_weight(shape=(input_shape[-1],1)) # matriz de pesos para opciones de control
        self.bt = self.add_weight(shape=(1,)) # bias para opciones de control

    
    def call(self, Xs):
        yopt = tf.keras.activations.softmax(tf.matmul(Xs, self.Wopt) + self.bopt)
        yt = (self.tf-self.t0)*tf.keras.activations.sigmoid(tf.matmul(Xs, self.Wt) + self.bt) + self.t0

        ys = tf.concat([yopt, yt], axis=-1)
        
        return ys


class Control(tf.keras.Model):
    def __init__(self, n, u1=16, u2=32, opts=4):
        super().__init__()

        self.n = n
        self.u1 = u1
        self.u2 = u2
        self.opts = opts
        
        self.d1 = tf.keras.layers.Dense(units=u1, activation='relu')
        self.d2 = tf.keras.layers.Dense(units=u2, activation='relu')
        self.trafic = TraficLights(opts=opts)
    
    def call(self, Xs):
        ys = self.d1(Xs)
        ys = self.d2(ys)
        ys = self.trafic(ys)

        return ys
    
    def get_gen(self):
        gen = []
        for layer in [self.d1, self.d2, self.trafic]:
            Ws = layer.get_weights()
            for wi in Ws:
                wi = np.array (wi)
                wi = wi.flatten(order='C')
                gen += list(wi)
        print(gen)
        return np.array(gen)
    
    def set_phen(self, gen): # 80
        aux = 0 # limite inferior para saber que variables tomar
        for layer in [self.d1, self.d2, self.trafic]:
            Ws = layer.get_weights()
            new_ws = []
            for wi in Ws:
                inc = np.prod(wi.shape)
                auxW = gen[aux:aux+inc]
                auxW = np.reshape(auxW, newshape=wi.shape)
                new_ws.append(auxW)
                aux += inc
            layer.set_weights(new_ws)

In [3]:
n = 4 # numero de variables de entrada consideradas
model = Control(n)

aux = np.ones((1,4), dtype=np.float32)
pred = model.predict(aux)

model.summary()

1/1 [==============================] - 16s 16s/step
Model: "control"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  80        
                                                                 
 dense_1 (Dense)             multiple                  544       
                                                                 
 trafic_lights (TraficLights  multiple                 165       
 )                                                               
                                                                 
Total params: 789
Trainable params: 789
Non-trainable params: 0
_________________________________________________________________
